### Air Quality Tensor
* `<date> <location> <air pollutants> (measurement)`
* Beijing Air Quality
  * 2,454,305 out of 2,524,536 (35,063 * 12 * 6)
* Korea Air Quality
  * 11,270,028 out of 18,368,364 (9,478 * 323 * 6)
* Madrid Air Quality
  * 8,036,759 out of 21,587,328 (64,248 * 24 * 14)

In [1]:
import csv
import time
import numpy as np
import pandas as pd

In [ ]:
beijing_df = pd.read_csv('../Data/air_quality/BeijingAirQuality/beijing.tensor', delimiter='\t', header=None)
korea_df = pd.read_csv('../Data/air_quality/KoreaAirQuality/korea_airquality.tensor', delimiter='\t', header=None)
madrid_df = pd.read_csv('../Data/air_quality/MadridAirQuality/1hour_madrid.tensor', delimiter='\t', header=None)

In [40]:
def get_tensor(df):
    start = time.time()

    dims = df[[0,1,2]].max()+1
    tensor = np.empty(dims) * np.nan
    tensor.shape

    for i, row in df.iterrows():
        indices = [[index] for index in np.int64(np.asarray(row[:-1]))]
        tensor[tuple(indices)] = np.double(row[3])

    avg = []
    for i in range(tensor.shape[2]):
        avg.append(np.nanmean(tensor[:,:,i]))
    inds = np.where(np.isnan(tensor))
    for ind in zip(inds[0], inds[1], inds[2]):
        tensor[ind] = avg[ind[-1]]
        
    print(time.time() - start)
    return tensor

In [ ]:
beijing_tensor = get_tensor(beijing_df)
korea_tensor = get_tensor(korea_df)
madrid_tensor = get_tensor(madrid_df)

349.54751777648926
2572.3920905590057


In [ ]:
np.where(np.isnan(beijing_tensor))
np.where(np.isnan(korea_tensor))
np.where(np.isnan(madrid_tensor))

In [ ]:
print(beijing_tensor.shape)
print(korea_tensor.shape)
print(madrid_tensor.shape)

### Hall from OLSTEC
* `hall1-200.mat`: 144 * 176 * 200
* `hall_144x100_frame2900-3899_pan.mat`: 144 * 100 * 1000

In [11]:
# for sample video
from cv2 import VideoWriter, VideoWriter_fourcc, imshow

def make_video(tensor, filename, isColor=True):
    start = time.time()
    height = tensor.shape[1]
    width = tensor.shape[2]
    FPS = 24

    fourcc = VideoWriter_fourcc(*'MP42')
    video = VideoWriter(filename, fourcc, float(FPS), (width, height), isColor)

    for frame in tensor:
        video.write(np.uint8(frame))
    video.release()
    print('created', filename, time.time()-start)

In [9]:
from scipy.io import loadmat
hall1 = loadmat('../Data/hall/hall1-200.mat')['XO']
hall1 = np.moveaxis(hall1, -1, 0)
hall1 = hall1.reshape(200, 144, 176, order='F')
# hall1.shape

make_video(hall1, 'hall1.avi', False)

created hall1.avi 0.20291566848754883


In [10]:
import h5py
with h5py.File('../Data/hall/hall_144x100_frame2900-3899_pan.mat', 'r') as f:
    hall2 = np.array(f['X0'])
    hall2 = hall2.reshape(1000, 144, 100, order='F')
    make_video(hall2, 'hall2.avi', False)

created hall2.avi 0.5112717151641846
